In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix, hstack
from keras.utils import np_utils
from sklearn.model_selection import KFold
import pickle

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
k_fold = False
if k_fold:
    pro_output = '../10fold_pro/10foldprob_lr_baseline.csv'
else:
    pro_output = '../pro/prob_lr_baseline2.csv'
output = '../output/prob_lr_baseline2.csv'
modify_tifi = False
tifi_dir = '../feature/tifi_min4.pkl'

In [3]:
t1=time.time()
train = pd.read_csv('../input/new_data/train_set.csv')
test = pd.read_csv('../input/new_data/test_set.csv')
test_id = pd.read_csv('../input/new_data/test_set.csv')[["id"]].copy()

In [4]:
y=(train["class"]-1).astype(int)

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [5]:
if modify_tifi:
    column="word_seg"
    vec = TfidfVectorizer(ngram_range=(1,2),min_df=4, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
    trn_term_doc = vec.fit_transform(train[column])
    test_term_doc = vec.transform(test[column])
    with open(tifi_dir,'wb') as fout:
        pickle.dump([trn_term_doc,test_term_doc],fout)
else:
     trn_term_doc, test_term_doc= pickle.load(open(tifi_dir,'rb'))

In [6]:
def kf_train(fold_cnt=3,rnd=1):
    now_nfold=0
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((102277,19)),np.zeros((102277,19))
    for train_index, test_index in kf.split(train):
        now_nfold+=1
        print ("now is {} fold".format(now_nfold))
        curr_x1 = trn_term_doc[train_index]
        hold_out_x1=trn_term_doc[test_index]
        
        curr_y, hold_out_y = y[train_index], y[test_index]
        clf = LogisticRegression(C=4, dual=True)
        clf.fit(curr_x1, curr_y)
        y_test = clf.predict_proba(test_term_doc)
        test_pred += y_test
        hold_out_pred = clf.predict_proba(hold_out_x1)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    return train_pred, test_pred

In [7]:
%%time
import time
import pickle
model_time = time.time()
print("start")
if k_fold:
    train_pred,test_pred = kf_train(fold_cnt=10,rnd=4)
    print (train_pred.shape,test_pred.shape)
    print ("[{}] finished nn model".format((time.time()-model_time)/3600))
    val_result = np.argmax(train_pred,axis=1)
    print ('score is {}'.format(f1_score(y, val_result, average='macro')))
else:
#     X_train, X_valid, Y_train, Y_valid = train_test_split(trn_term_doc, y, test_size=0.15, random_state=42)
    clf = LogisticRegression(C=4, dual=True)
    clf.fit(trn_term_doc, y)
#     vaild = clf.predict_proba(X_valid)
#     val_result = np.argmax(vaild,axis=1)
#     print ('score is {}'.format(f1_score(Y_valid, val_result, average='macro')))
    test_pred = clf.predict_proba(test_term_doc)

start
CPU times: user 5min 49s, sys: 356 ms, total: 5min 49s
Wall time: 5min 49s


In [8]:
train_pred = clf.predict_proba(trn_term_doc)

In [9]:
with open('../feature/stackint_pkl_file/lr.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

In [10]:
test_prob=pd.DataFrame(test_pred)
test_prob.columns=["class_prob_%s"%i for i in range(1,20)]
test_prob["id"]=list(test_id["id"])
test_prob.to_csv(pro_output,index=None)

In [9]:
preds=np.argmax(test_pred,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["class"]
test_pred["class"]=(test_pred["class"]+1).astype(int)
print(test_pred.shape)
print(test_id.shape)
test_pred["id"]=list(test_id["id"])
test_pred[["id","class"]].to_csv(output,index=None)
t2=time.time()
print("time use:",t2-t1)

(102277, 1)
(102277, 1)
time use: 3140.9321308135986
